In [12]:
import argparse
import json
import logging
import random
from collections import defaultdict, OrderedDict
from pathlib import Path

import numpy as np
import torch
import torch.utils.data
from experiments.pfedhn_pc.utils import get_average_model, weighted_aggregate_model
# from tqdm import trange

from experiments.pfedhn_pc.models import CNNHyperPC, CNNTargetPC, CNNTargetPC_M, LocalLayer
from experiments.pfedhn_pc.node import BaseNodesForLocal, BaseNodesForLocals_M
from experiments.utils import get_device, set_logger, set_seed, str2bool

import copy
from model import *

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
data_name = 'cifar10'
data_path = 'data'
num_nodes = 50
num_steps = 5000
inner_steps = 50
optim = 'sgd'
lr = 5e-2
inner_lr = 5e-3 
embed_lr = None
wd = 1e-3
inner_wd = 5e-5
embed_dim = -1
batch_size = 
eval_every = 
save_path = 

args = {
    'data_name' = 

}

SyntaxError: invalid syntax (Temp/ipykernel_317040/3759357367.py, line 14)

In [ ]:
def eval_model(nodes, num_nodes, inet, net, criteria, device, split):
    curr_results = evaluate(nodes, num_nodes, inet, net, criteria, device, split=split)
    total_correct = sum([val['correct'] for val in curr_results.values()])
    total_samples = sum([val['total'] for val in curr_results.values()])
    avg_loss = np.mean([val['loss'] for val in curr_results.values()])
    avg_acc = total_correct / total_samples

    all_acc = [val['correct'] / val['total'] for val in curr_results.values()]

    return curr_results, avg_loss, avg_acc, all_acc


@torch.no_grad()
def evaluate(nodes: BaseNodesForLocal, num_nodes, inet, net, criteria, device, split='test'):
    inet.eval()
    results = defaultdict(lambda: defaultdict(list))

    for node_id in range(num_nodes):  # iterating over nodes

        running_loss, running_correct, running_samples = 0., 0., 0.
        if split == 'test':
            curr_data = nodes.test_loaders[node_id]
        elif split == 'val':
            curr_data = nodes.val_loaders[node_id]
        else:
            curr_data = nodes.train_loaders[node_id]

        weights = inet(torch.tensor([node_id], dtype=torch.long).to(device))
        net.load_state_dict(weights)

        for batch_count, batch in enumerate(curr_data):
            img, label = tuple(t.to(device) for t in batch)
            net_out = net(img)
            pred = nodes.local_layers[node_id](net_out)
            running_loss += criteria(pred, label).item()
            running_correct += pred.argmax(1).eq(label).sum().item()
            running_samples += len(label)

        results[node_id]['loss'] = running_loss / (batch_count + 1)
        results[node_id]['correct'] = running_correct
        results[node_id]['total'] = running_samples

    return results

In [ ]:
a[1]

3